In [164]:


from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition



In [165]:


from idaes.core import FlowsheetBlock



In [166]:


from idaes.models.unit_models import (PressureChanger,
                                      Mixer,
                                      Separator as Splitter,
                                      Heater,
                                      StoichiometricReactor,
                                      HeatExchanger, Compressor)


In [167]:
from idaes.models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom

# Import idaes logger to set output levels
import idaes.logger as idaeslog

In [168]:
model = ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)

In [169]:
model.fs.properties = iapws95.Iapws95ParameterBlock()

import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props



model.fs.thermo_params = thermo_props.HDAParameterBlock()
model.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        property_package=model.fs.thermo_params)



model.fs.heater = Heater(property_package=m.fs.thermo_params,
                   has_pressure_change=False,
                   has_phase_equilibrium=True)

model.fs.C101 = Compressor(property_package=model.fs.properties)

In [170]:
# Add a HX model to the flowsheet.
model.fs.H101 = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)


In [171]:
# Add a HX model to the flowsheet.
model.fs.H102 = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

In [172]:
# Add a HX model to the flowsheet.
model.fs.H103 = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

In [173]:
# Add a HX model to the flowsheet.
model.fs.H104 = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

In [174]:

# Air stream (tube side): H101 → H102
model.fs.a01 = Arc(source=model.fs.H101.tube_outlet, destination=model.fs.H102.tube_inlet)

# Product gas stream (shell side): H104 → H103 (flowing right to left)
model.fs.a02 = Arc(source=model.fs.H104.shell_outlet, destination=model.fs.H103.shell_inlet)

In [175]:


TransformationFactory("network.expand_arcs").apply_to(m)



In [176]:
# ---- PROPAGATE STATE (Optional) ----
from idaes.core.util.initialization import propagate_state
propagate_state(model.fs.a01)
propagate_state(model.fs.a02)

In [181]:
print(degrees_of_freedom(model))

32


In [ ]:
# ---- FIXING VARIABLES ----
# Fix the flow rates for the heat exchangers
model.fs.H101.shell_inlet.flow_mol.fix(10)  # Molar flow rate in mol/s
model.fs.H102.tube_inlet.flow_mol.fix(10)   # Molar flow rate in mol/s

In [186]:

model.fs.H101.shell_outlet.temperature.fix(192.60)  # Temperature in C
model.fs.H101.tube_outlet.temperature.fix(496.94)   # Temperature in C

AttributeError: 'ScalarPort' object has no attribute 'temperature'

In [177]:
import idaes_ui

In [178]:
import idaes_ui.fv
dir(idaes_ui.fv)

['VisualizeResult',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'errors',
 'flowsheet',
 'fsvis',
 'icons',
 'model_server',
 'models',
 'persist',
 'validate_flowsheet',
 'visualize']

In [179]:
from idaes_ui.fv import visualize 

In [180]:
visualize(model.fs, "HeatExchangerNetwork")

2025-04-09 10:36:18 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 50189
2025-04-09 10:36:18 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerNetwork.json'
2025-04-09 10:36:18 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerNetwork.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-04-09 10:36:20 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:50189/app?id=HeatExchangerNetwork


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000029D1B70A680>, port=50189, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x0000029D187AD720>)